In [1]:
from picamera.array import PiYUVArray
from picamera import PiCamera

from scipy.signal import find_peaks, butter, filtfilt
from pwm import PWM
import tools
import threading
import ipywidgets as ipw
import time
#import matplotlib.pyplot as plt
#import skimage as ski

WIDTH_  = 224
HEIGHT_ = 160
CENTER_ = WIDTH_ // 2

# Camera resolution
res = (WIDTH_, HEIGHT_)

# Run a track detection algorithm on a single horizontal line.
# Uses YUV420 image format as the Y component corresponds to image intensity (gray image)
# and thus there is no need to convert from RGB to BW

camera = PiCamera()
        
# Check the link below for the combinations between mode and resolution
# https://picamera.readthedocs.io/en/release-1.13/fov.html#sensor-modes
camera.sensor_mode = 7
camera.resolution = res
camera.framerate = 120

# To filter the noise in the image we use a 3rd order Butterworth filter

# Wn = 0.02, the cut-off frequency, acceptable values are from 0 to 1
b, a = butter(3, 0.02)

pwm0 = PWM(0)
pwm1 = PWM(1)

pwm0.period = 20000000
pwm1.period = 20000000

pwm0.enable = True
pwm1.enable = True

In [2]:
pwm0.duty_cycle = 1000000
pwm1.duty_cycle = 1600000
time.sleep(1)
pwm1.duty_cycle = 1100000
time.sleep(1)
pwm1.duty_cycle = 2100000
time.sleep(1)
pwm1.duty_cycle = 1600000

In [3]:
#max speed that this algo works
#ucla = 1120000
#sonic = 1250000

UCLA_         =  1139000
USC_          =  1275000
SONIC_        =  1260000
LINE_         =  HEIGHT_ // 2
INTERVAL_     =  0.0
CENTER_THRESHOLD_ = WIDTH_ // 10

STRAIGHT_     =  1600000
RIGHT_        =  1100000
LEFT_         =  2100000
NUM_PREV_VAL_ =  8

killSwitch = ipw.ToggleButton(value=False, description='Shut it down')
minMaxSlider = ipw.IntRangeSlider(value=[UCLA_/1000,SONIC_/1000], min = 1000, max = 1500,step = 1, description = 'Speed Ranges')

display(minMaxSlider)
display(killSwitch)

def main_loop():
    rawCapture = PiYUVArray(camera, size=res)
    stream = camera.capture_continuous(rawCapture, format="yuv", use_video_port=True)

    #for pwm1
    #duty_cycle

    #      (array([482, 542], dtype=int32), {'peak_heights': array([186.831036  , 185.01286929])})

    numFrames = 0

    #if linePos is 1 then the line is to the right of the car
    #if linePos is -1 then the line is to the left of the car
    #if linePos is 0 either uhoh we're doomed
    #if lost is true then the line is lost
    linePos = 0
    lost = False
    prevSteerDC = pwm1.duty_cycle
    prevSteerDiffArr = []
    prevPeakAvgArr = []
    for i in range(NUM_PREV_VAL_):
        prevSteerDiffArr.append(-1)
        prevPeakAvgArr.append(-1)
    
    for f in stream:
        t = time.time()
        #speeds = update_speeds()
        
        #UCLA_ = speeds[0]
        #SONIC_ = speeds[1]
        
        if killSwitch.value:
            break

        numFrames += 1
        # Get the intensity component of the image (a trick to get black and white images)
        I = f.array[:, :, 0]
        # Reset the buffer for the next image
        rawCapture.truncate(0)
        # Select a horizontal line in the middle of the image
        L = I[LINE_, :]
        # Smooth the transitions so we can detect the peaks 
        Lf = filtfilt(b, a, L)
        # Find peaks which are higher than 0.5
        p = find_peaks(Lf, height=128)

        totalPeakAmt = 0
        avgPeakLoc = -520
        numPeaks = 0

        for peak in p[0]:
            numPeaks += 1
            totalPeakAmt += peak

        if numPeaks != 0:
            avgPeakLoc = totalPeakAmt / numPeaks
            if numPeaks == 3:
                print("THIS IS THE BEGINNING OF THE END")

        if avgPeakLoc != -520 :
            if avgPeakLoc < CENTER_:
                if lost and linePos == 1:
                    #pwm1.duty_cycle = RIGHT_
                    #continue
                    break
                linePos = -1
            else:
                if lost and linePos == -1:
                    #pwm1.duty_cycle = LEFT_
                    #continue
                    break
                linePos = 1

            pwm1.duty_cycle = int(STRAIGHT_ - ((avgPeakLoc - CENTER_)/CENTER_)*500000)
            lost = False
        else:
            if linePos == -1:
                pwm1.duty_cycle = LEFT_
            elif linePos == 1:
                pwm1.duty_cycle = RIGHT_
            else:
                pwm1.duty_cycle = STRAIGHT_

            lost = True
            #pwm0.duty_cycle = SONIC_

        if abs(prevSteerDC - STRAIGHT_) < abs(pwm1.duty_cycle - STRAIGHT_) and prevSteerDiffArr[0] > 0 and prevSteerDiffArr[1] > 0 and abs(pwm1.duty_cycle - STRAIGHT_) > 400000:
            pwm0.duty_cycle = SONIC_
            #prevSteerDiffArr[0] = -1
            '''+ (500000 - abs(pwm1.duty_cycle - STRAIGHT_)))'''
        elif abs(avgPeakLoc - CENTER_) <= CENTER_THRESHOLD_:
            i = 0
            while i < NUM_PREV_VAL_:
                if abs(prevPeakAvgArr[i] - CENTER_) > CENTER_THRESHOLD_:
                    break
                i += 1
            #print("STRAIGHT")
            if i == NUM_PREV_VAL_:
                pwm0.duty_cycle = USC_
        else:
            pwm0.duty_cycle = UCLA_
            if abs(pwm1.duty_cycle - STRAIGHT_) > 300000:
                pwm0.duty_cycle = UCLA_ + 4000

        i = NUM_PREV_VAL_ - 1
        while i > 0:
            prevSteerDiffArr[i] = prevSteerDiffArr[i - 1]
            prevPeakAvgArr[i] = prevPeakAvgArr[i - 1]
            i -= 1
        
        prevSteerDiffArr[0] = abs(pwm1.duty_cycle - STRAIGHT_) - abs(prevSteerDC - STRAIGHT_)
        prevPeakAvgArr[0] = avgPeakLoc

        prevSteerDC = pwm1.duty_cycle

        time_elapsed = time.time() - t
        
        print(prevPeakAvgArr)
        print(prevSteerDiffArr)
        print(sum(prevSteerDiffArr))
        print(sum(prevSteerDiffArr) / len(prevSteerDiffArr))
        print(pwm1.duty_cycle)
        print("Elapsed {:0.4f} seconds, estimated FPS {:0.2f}".format(time_elapsed, 1 / time_elapsed))
        time.sleep(INTERVAL_)
        
    pwm0.duty_cycle = 1000000

    # Release resources
    stream.close()
    rawCapture.close()
    camera.close()

IntRangeSlider(value=(1139, 1260), description='Speed Ranges', max=1500, min=1000)

ToggleButton(value=False, description='Shut it down')

In [4]:
def update_speeds():
    UCLA_ = minMaxSlider.value[0] * 1000
    SONIC_ = minMaxSlider.value[1] * 1000
    return [UCLA_, SONIC_]

In [5]:
#minMaxSlider.observe(update_speeds)

In [6]:
t = threading.Thread(target=main_loop)
t.start()

In [7]:
'''plt.imshow(I)'''

'plt.imshow(I)'

[113.0, -1, -1, -1, -1, -1, -1, -1]
[206250, -1, -1, -1, -1, -1, -1, -1]
206243
25780.375


In [2]:
'''plt.plot(L, label="raw")
plt.plot(Lf, label="filtered")
plt.ylim([0, 300])
plt.legend()'''

NameError: name 'plt' is not defined

In [1]:
camera.close()

NameError: name 'camera' is not defined

In [7]:
pwm0.duty_cycle = 1000000

In [1]:
from pwm import PWM

pwm0 = PWM(0)

pwm0.duty_cycle = 1000000

pwm0.period = 20000000

pwm0.enable = True